In [1]:
# For debug mode
%load_ext autoreload
%autoreload 2

In [1]:
wig_list_paths = [
    "../database/company_list/wig_chemia.csv",
    "../database/company_list/wig_odziez.csv",
    "../database/company_list/wig_spozywczy.csv",
]

### Scraping financial report

In [39]:
import pandas as pd
import scripts as ds

for wig_path in wig_list_paths:
    company_list = pd.read_csv(wig_path)["TICKER"].to_list()
    ds.Scraper(ds.FinancialReport(company_list)).scrape()

### Scraping stock price

In [18]:
import yfinance as yf
import pandas as pd

STOCK_PATH = "../database/stock_price/"

for wig_path in wig_list_paths:
    company_list = pd.read_csv(wig_path)["TICKER"].to_list()
    for ticker in company_list:
        yaho = yf.Ticker("{}.WA".format(ticker))
        history = pd.DataFrame(yaho.history(period="max"))
        history.to_csv("{}{}.csv".format(STOCK_PATH, ticker))  

In [16]:
import pickle
import pandas as pd

DATE_PATH = "../database/financial_report/date/datePublicationDict"

with open(DATE_PATH, 'rb') as f:
    dict = pickle.load(f)

print(dict.keys())

for wig_path in wig_list_paths:
    company_list = pd.read_csv(wig_path)["NAME"].to_list()
    for name in company_list:
        print(dict[name]["Raport roczny"])



dict_keys(['PKO BP SA', 'Inter Cars SA', 'Bank Handlowy w Warszawie SA', 'Rawlplug SA', 'Wielkopolska Wytwórnia Żywności PROFI SA', 'FAM SA', 'Mediatel SA', 'Capital Partners SA', 'KCI SA', 'Alchemia SA', 'TONSIL SA w upadłości', 'ATM Grupa SA', 'Praterm SA', 'Grupa Onet.pl SA', 'PBG SA  w restrukturyzacji w likwidacji', 'DGA SA', 'ENERGOMONTAŻ-POŁUDNIE SA', 'Instalexport SA', 'Bytom SA', 'Bank Millennium SA', 'BOŚ SA', 'Mondi Świecie SA', 'Polcolorit SA', 'Iławskie Zakłady Drobiarskie EKODROB SA', 'UNIMIL SA', 'PMPG Polskie Media SA', 'Elektrobudowa SA  w upadłości', 'Betacom SA', 'Kompap SA', 'Ferrum SA', 'mBank SA', 'Zakłady Przemysłu Cukierniczego MIESZKO SA', 'INTERIA SA', 'Wasko SA', 'Stalprodukt SA', 'Comp SA', 'Swissmed Centrum Zdrowia SA', 'Advadis SA w upadłości likwidacyjnej', 'Elstar Oils SA', 'Orbis SA', 'Orange Polska SA', 'Rafako SA', 'MDI Energia SA', 'TIM SA', 'Stormm SA', 'MNI SA', 'SOKOŁÓW SA', 'Kogeneracja SA', 'Energoaparatura SA', 'Relpol SA', 'VRG SA', 'KB Dom SA

KeyError: 'PEPEES SA'